# HNLQ: Part-02

This series of notebook demonstrates hierarchical nested lattice quantization using the D4 lattice, showing how multi-level quantization can improve rate-distortion performance.

In part-01, we primarily check the methods, and see it side-by-side by with NLQ.

In [1]:
import sys
sys.path.append('../../')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from gemvq.quantizers.hnlq import HNLQ as HNLQ
from gemvq.quantizers.hnlq import HNLQConfig as QuantizerConfig


In [2]:
%load_ext autoreload
%autoreload 2

An important observation. In terms API and implementation HNLQ and NLQ are very similar. In fact, NLQ is special case of HNLQ with M=1.  Therefore, whatever we observed for NLQ, we should observe for NLQ with M=1.

The NLQ API is a strict subset of HNLQ API. That is what we will see below, as sanity check.

### D4 Lattice via Config

In [3]:
# create a quantizer for D4 lattice using QuantizerConfig object

# Method 1: Using QuantizerConfig object (most explicit)
config = QuantizerConfig(
    lattice_type='D4',      # Automatically loads D4 lattice components
    q=2,                    # Quantization parameter (alphabet size)
    beta=1.0,              # Scaling parameter for quantization
    alpha=0.5,             # Scaling parameter for overload handling
    eps=1e-8,              # Small perturbation parameter
    overload=True,          # Handle overload by scaling
    max_scaling_iterations=10,
    with_tie_dither=True,   # Add dither for tie breaking
    with_dither=False,       # No randomized dither    ,
    M =1,
    decoding='full'
)

quantizer = HNLQ(config)

def print_quantizer_summary(quantizer):
    d = len(quantizer.G)
    x = np.random.randn(d)
    x = x/np.sqrt(np.linalg.norm(x))
    print("Original vector:", x)


    x_hat = quantizer.quantize(x)
    

    enc, T = quantizer.encode(x)
    x_dq = quantizer.decode(enc, T)

    
    print(f"Lattice type: {quantizer.lattice_type}")
    print(f"Input: {x}")
    print(f"Encoded: {enc}")
    print(f"Scaling iterations: {T}")
    print(f"Reconstructed: {x_hat}")
    print(f"Error: {np.mean((x - x_hat)**2):.6f}")


print_quantizer_summary(quantizer)

Original vector: [ 0.05155419 -0.56711165  0.71035388  1.03790839]
Lattice type: D4
Input: [ 0.05155419 -0.56711165  0.71035388  1.03790839]
Encoded: (array([1, 1, 0, 1]),)
Scaling iterations: 0
Reconstructed: [0. 0. 1. 1.]
Error: 0.102401


### D4 Lattice via Dict

In [4]:
# Create configuration as dictionary
config_dict = {
    'lattice_type': 'D4',
    'q': 3,
    'beta': 1.0,
    'alpha': 1.0,
    'eps': 1e-8,
    'M': 3,
    'overload': True,
    'decoding': 'coarse_to_fine',
    'max_scaling_iterations': 10,
    'with_tie_dither': True,
    'with_dither': False
}

# Create HNLQ with configuration dictionary
quantizer = HNLQ(config_dict)
print_quantizer_summary(quantizer)


Original vector: [-0.22042875  0.13826275  0.80607687  0.60639457]
Lattice type: D4
Input: [-0.22042875  0.13826275  0.80607687  0.60639457]
Encoded: (array([2, 2, 1, 2]), array([0, 0, 0, 0]), array([0, 0, 0, 0]))
Scaling iterations: 0
Reconstructed: [0. 0. 1. 1.]
Error: 0.065059


### E2 lattice via specical constructor

In [5]:
# Create Z² hierarchical quantizer
quantizer = HNLQ.create_z2_quantizer(
    q=3,
    M=3,
    beta=1.0,
    alpha=1.0,
    eps=1e-8,
    overload=True,
    decoding='full',
    with_tie_dither=True,
    with_dither=False
)
print_quantizer_summary(quantizer)

Original vector: [-0.57185268 -1.11180512]
Lattice type: Z2
Input: [-0.57185268 -1.11180512]
Encoded: (array([2, 2]), array([0, 0]), array([0, 0]))
Scaling iterations: 0
Reconstructed: [-1. -1.]
Error: 0.097905


### Custom Lattice

In [6]:
# Define custom generator matrix and closest point function
G_custom = np.array([[1, 0], [0, 1]])  # 2x2 identity matrix
def custom_closest_point(x):
    return np.floor(x + 0.5)

# Create configuration
config = QuantizerConfig(
    lattice_type='Z2',  # Can be any supported type
    q=3,
    M=2,
    beta=1.0,
    alpha=1.0,
    eps=1e-8
)

# Create HNLQ with custom components
quantizer = HNLQ(config, G=G_custom, Q_nn=custom_closest_point)
print_quantizer_summary(quantizer)

Original vector: [-0.88858375 -0.36374477]
Lattice type: Z2
Input: [-0.88858375 -0.36374477]
Encoded: (array([2, 0]), array([0, 0]))
Scaling iterations: 0
Reconstructed: [-1.  0.]
Error: 0.072362


### Minimal Configuration

In [7]:
# Minimal configuration with defaults
config_minimal = QuantizerConfig(
    lattice_type='D4',
    q=3,
    M=3
    # All other parameters use defaults
)
quantizer = HNLQ(config_minimal)
print_quantizer_summary(quantizer)


Original vector: [-0.14730948  0.54727923 -1.20288527 -0.16084299]
Lattice type: D4
Input: [-0.14730948  0.54727923 -1.20288527 -0.16084299]
Encoded: (array([0, 0, 1, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]))
Scaling iterations: 0
Reconstructed: [ 0.  1. -1.  0.]
Error: 0.073422


### Different decoding strategies

In [8]:
# Full decoding (default)
hnlq_full = HNLQ.create_d4_quantizer(q=3, M=3, decoding='full')

# Coarse-to-fine decoding
hnlq_coarse = HNLQ.create_d4_quantizer(q=3, M=3, decoding='coarse_to_fine')

# Progressive decoding
hnlq_progressive = HNLQ.create_d4_quantizer(q=3, M=3, decoding='progressive')

### Advanced configurations

In [9]:
# Advanced configuration with all parameters specified
config_advanced = QuantizerConfig(
    lattice_type='E8',
    q=4,
    beta=0.5,
    alpha=1.5,
    eps=1e-10,
    M=4,
    overload=False,
    decoding='progressive',
    max_scaling_iterations=20,
    with_tie_dither=False,
    with_dither=True
)

hnlq_advanced = HNLQ(config_advanced)

## A simple example
of using hierarchical lattice quantization to quantize a 4D vector.

We will take $x=(1.2,-0.7,2.4,0.3)$

In [10]:
np.random.seed(31)
q = 2
M = 3

config = QuantizerConfig(lattice_type='D4', q=q, M=M)
hq = HNLQ(config=config)
    

# Create test sample and normalize it
test_sample = np.array([1.2, -0.7, 2.4, 0.3])
test_sample = test_sample/np.linalg.norm(test_sample)  # normalize to unit vector
print("Original vector:", test_sample)

# Test quantization
x_q = quantizer.quantize(test_sample)
print("Quantized vector:", x_q)

# Test encode/decode separately
enc, T = quantizer.encode(test_sample)
x_dq = quantizer.decode(enc, T)
print("encoded:", enc)
print("overloading factor:", T)
print("Decoded vector:", x_dq)

# Print error
error = np.linalg.norm(test_sample - x_dq)
print(f"Reconstruction Error (L2 norm): {error:.6f}")

Original vector: [ 0.43022084 -0.25096216  0.86044169  0.10755521]
Quantized vector: [1. 0. 1. 0.]
encoded: (array([2, 0, 2, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]))
overloading factor: 0
Decoded vector: [1. 0. 1. 0.]
Reconstruction Error (L2 norm): 0.647051
